## halo mass

In [ ]:
import numpy as np
import illustris_python as il

basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
fields   = ['GroupFirstSub','Group_M_Crit200','GroupPos']

h = 0.677

f = 10**(-2) / h  #unit: 10**12 M_{\odot}

FoFGroup = il.groupcat.loadHalos(basePath, snapNum, fields)
ID_group = np.where((FoFGroup['Group_M_Crit200'] * f>0.3) & (FoFGroup['Group_M_Crit200'] * f<3))
ID_neb   = np.where(FoFGroup['Group_M_Crit200'] * f>0.5)

ID = FoFGroup['GroupFirstSub'][ID_group]
    

## stellar mass of central galaxy

In [ ]:
fields = ['SubhaloPos','SubhaloMassInRadType']
subhalos = il.groupcat.loadSubhalos(basePath, snapNum, fields)
fs = 1 / h #unit: 10**10 M_{\odot}

index = np.where((subhalos['SubhaloMassInRadType'][ID,4] * fs>1) 
                 & (subhalos['SubhaloMassInRadType'][ID,4] * fs<10))

#ID_S       = ID[index]
ID_group   = ID_group[0][index]

## isolation

In [ ]:
def periodDistance(distance,L = 35000):
    if distance > L/2 :
        return  distance - L
    else:
        if distance < -L/2 :
            return distance + L
        else:
            return distance

In [ ]:
def isolation(Id,ID_neb,a=1):
    Rad = np.zeros(0)
    dis = np.zeros(3)
    h = 0.677
    
    for i in ID_neb[0]:
        if Id == i:
            continue

        for j in np.arange(3):
            d = FoFGroup['GroupPos'][Id,j]-FoFGroup['GroupPos'][i,j]  # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            dis[j] = cDistance * a /h   # unit kpc
        Rad = np.append(Rad, np.linalg.norm(dis))
        
    if len(np.where(Rad <= 780)[0]) == 0:
        return True
    else:
        return False

In [ ]:
ID_G_iso = np.zeros(0,dtype=int)
for Id in ID_group:
    if isolation(Id,ID_neb=ID_neb):
        ID_G_iso = np.append(ID_G_iso,Id)
ID_S = FoFGroup['GroupFirstSub'][ID_G_iso]        

## satellite num criteria 

In [ ]:
def ratio_axis(sat_class_pos):
    N = sat_class_pos.shape[0]
    M = np.zeros((3,3))
    r_0 = np.zeros((1,3))
    
    r_0[0,0] = np.mean(sat_class_pos[:,0])
    r_0[0,1] = np.mean(sat_class_pos[:,1])
    r_0[0,2] = np.mean(sat_class_pos[:,2])
    
    sat_class_pos -= r_0
    
    for i in np.arange(N):
        v = np.array([sat_class_pos[i,:]])
        M += np.dot(v.T,v)
        
    eigval,eigvec = np.linalg.eig(M)
    index_mindir  = np.argsort(eigval)[0]
    
    return np.sqrt(np.min(eigval)/np.max(eigval)), eigvec[:,index_mindir]

In [ ]:
def isotropic_pos(N):
    
    import numpy as np
    
    u = np.random.normal(0,1,N)
    v = np.random.normal(0,1,N)
    w = np.random.normal(0,1,N)
    
    norm = np.sqrt(u**2 + v**2 + w**2)
    
    x = u / norm
    y = v / norm
    z = w / norm
    
    return np.vstack((x,y,z)).T

In [ ]:
def coherent_iso_fast(sat_class_pos,sat_class_vel,N):
    import itertools
    
    L = sat_class_pos.shape[0]
    sat_class_ang     = np.zeros((L,3))
    #sat_class_ang_norm = np.zeros((L,3))
    for i in np.arange(L):
        sat_class_ang[i,:] = np.cross(sat_class_pos[i,:],sat_class_vel[i,:])
        sat_class_ang[i,:] = sat_class_ang[i,:] / np.linalg.norm(sat_class_ang[i,:])
    
    indexes = np.arange(L)
    index_comb  = np.array(list(itertools.combinations(indexes,N)))
    
    MEANS = isotropic_pos(1000)
    means_1 = np.zeros((0,3))
    RMSs_1  = np.zeros(0)
    means = np.zeros((0,3))
    RMSs  = np.zeros(0)
    
    for mean in MEANS:
        RMS  = np.abs(np.dot(sat_class_ang,mean))
        #RMS  = np.dot(sat_class_ang[com,:],mean)
        RMS  = np.mean(np.square(np.arccos(RMS)))
        RMS  = np.sqrt(RMS)

        means_1 = np.append(means_1,np.array([mean]),axis=0)
        RMSs_1  = np.append(RMSs_1,RMS)
        
    index = np.argsort(RMSs_1)[0]
    MEAN  = means_1[index]
    
    for i in np.arange(L):
        cos_theta = np.dot(sat_class_ang[i],MEAN)
        if cos_theta<0:
            sat_class_ang[i] = sat_class_ang[i] * (-1)
    
    for com in index_comb:
        mean = np.mean(sat_class_ang[com,:],axis=0)
        mean/= np.linalg.norm(mean)
#         RMS  = np.abs(np.dot(sat_class_ang[com,:],mean))
        RMS  = np.dot(sat_class_ang[com,:],mean)
        RMS  = np.mean(np.square(np.arccos(RMS)))
        RMS  = np.sqrt(RMS)
        
        means = np.append(means,np.array([mean]),axis=0)
        RMSs  = np.append(RMSs,RMS)
        
    index = np.argsort(RMSs)[0]
    
    return np.rad2deg(RMSs[index]),means[index],index_comb[index]
    
    

In [ ]:
def PosVel_no_FOF(haloID,basePath,snapNum,N=11,M=300):
    import numpy as np
    import illustris_python as il
    from functools import reduce
    
    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    
    snapNum_fst = snapNum
    SNums=[snapNum]
    
    h = 0.677
    a = 1 / (1+SnapRedshift[snapNum])
    f1 = 10**10 / h
    f2 = a / h
    
    
    fields = ['SubhaloMassInRadType','SubhaloPos','SubhaloVel','SubhaloFlag']
    
    FoFGroup    = il.groupcat.loadSingle(basePath, snapNum, haloID)
#     subHaloData = il.groupcat.loadSubhalos(basePath, snapNum, fields)
    
    ID_firstSub = FoFGroup['GroupFirstSub']
    
    r_0   = M * h
    
    satID = fun_dm(haloID,SNums,r_0,snapNum_fst)[0]
    
    NumSub  = len(satID)
    
    if NumSub < N:
        print(0)
        return [0],[0],[0],[0],[0]
    
#     satID = np.arange(NumSub-1) + ID_firstSub + 1
    
    sat_Mstar = np.zeros(0)
    sat_pos   = np.zeros((0,3))
    sat_vel   = np.zeros((0,3))
    sat_flag  = np.zeros(0)
    
    for id in satID:
        Data = il.groupcat.loadSingle(basePath, snapNum, subhaloID=id)
        
        sat_Mstar = np.append(sat_Mstar,Data['SubhaloMassInRadType'][4]*f1)
        sat_pos   = np.append(sat_pos,Data['SubhaloPos'].reshape((1,3)),axis=0)
        sat_vel   = np.append(sat_vel,Data['SubhaloVel'].reshape((1,3)),axis=0)
        sat_flag  = np.append(sat_flag,Data['SubhaloFlag'])
        
    Data = il.groupcat.loadSingle(basePath, snapNum, subhaloID=ID_firstSub)
    
    cen_pos = Data['SubhaloPos']
    cen_vel = Data['SubhaloVel']
#     sat_Mstar = subHaloData['SubhaloMassInRadType'][satID,4] * f1
#     sat_pos   = subHaloData['SubhaloPos'][satID,:] 
#     sat_vel   = subHaloData['SubhaloVel'][satID,:]
#     sat_flag  = subHaloData['SubhaloFlag'][satID]
    
#     cen_pos   = subHaloData['SubhaloPos'][ID_firstSub,:]
#     cen_vel   = subHaloData['SubhaloVel'][ID_firstSub,:]
    
    for i in np.arange(NumSub):
        for j in np.arange(3):
            d = sat_pos[i,j]-cen_pos[j]  # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            sat_pos[i,j] = cDistance * f2   # unit kpc
            sat_vel[i,j] = sat_vel[i,j] - cen_vel[j] # change unit to km/s
            
#     sat_Rad = np.zeros(NumSub)

#     for i in np.arange(NumSub): #get the radii of every star and wind particle
#         sat_Rad[i] = np.linalg.norm(sat_pos[i,:])

    sat_Rad = np.linalg.norm(sat_pos,axis=1)
        
    
    sat_index = reduce(np.intersect1d,[np.where((sat_Rad <= 300)&(sat_Rad >= 15)),
                                       np.where(sat_Mstar > 10**5),
                                      np.where(sat_flag==1)])
    if len(sat_index) < N:
        print(1)
        return [0],[0],[0],[0],[0]
    
    sat_index_1 = np.argsort(sat_Mstar[sat_index])[-1:-(N+1):-1]
    sat_index = sat_index[sat_index_1]
    
    sat_class_pos = sat_pos[sat_index,:]
    sat_class_vel = sat_vel[sat_index,:]
    sat_class_ID  = satID[sat_index]
    
    return sat_class_pos,sat_class_vel,sat_class_ID,ID_firstSub,haloID
    

In [ ]:
def fun_dm(haloID,SNums,r_0,snapNum_fst=99):
    POS_dm=[]
    i = 0
    for num in SNums:
        pos_dm = pos_ori(haloID,num,r_0,snapNum_fst)
        POS_dm.append(pos_dm)
        i+=1
        #print(i)
        
    return POS_dm

In [ ]:
def pos_ori(haloID,snapNum,r_0,snapNum_fst=99):
    import numpy as np
    import illustris_python as il
    import random
    from functools import reduce

    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
    #SnapNum  = 99
    h = 0.677

    fields = ['SubhaloPos']
    fields_tree = ['SubhaloPos','SnapNum']

    subhaloID  = il.groupcat.loadSingle(basePath, snapNum_fst, haloID=haloID)['GroupFirstSub']
    cen_tree   = il.sublink.loadTree(basePath,snapNum_fst,id=subhaloID,fields=fields_tree,
                                               onlyMPB=True)
    
    index = np.where(cen_tree['SnapNum']==snapNum)[0][0]
    #snapNum = cen_tree['SnapNum'][index]
    a = 1 / (1+SnapRedshift[snapNum])
    #f = a / h
    f = 1 / h  # use comoving coordinate: ckpc

    cen_pos   = cen_tree['SubhaloPos'][index]
    par_pos   = il.groupcat.loadSubhalos(basePath, snapNum, fields)
    
    index_0 = selec_first(par_pos[:,0],cen_pos[0],r_0)
    index_1 = selec_first(par_pos[:,1],cen_pos[1],r_0)
    index_2 = selec_first(par_pos[:,2],cen_pos[2],r_0)
    
    index_first = reduce(np.intersect1d, [index_0, index_1, index_2])
    
#     if len(index_first) > 7500:
#         array = random.sample(range(len(index_first)),7500)
#         index_first = index_first[array]

    par_pos = par_pos[index_first,:]

    Num_pars = len(par_pos[:,0])

    for i in np.arange(Num_pars):
        for j in np.arange(3):
            d = par_pos[i,j]-cen_pos[j]                    # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            par_pos[i,j] = cDistance * f                   # unit ckpc
            
    norm = np.linalg.norm(par_pos,axis=1)
    if len(norm)!=len(par_pos[:,0]):
        raise Exception('erro')
    
    index = np.where((norm <= r_0*f)&(norm >= 15))[0]
    #par_pos = par_pos[index,:]
    
    return index_first[index]

In [ ]:
def selec_first(arr,x,r_0,L=35000):
    if x + r_0 > L:
        a = x+r_0-L
        b = x - r_0
        key = 0
    elif x - r_0 < 0:
        a = x+r_0
        b = x-r_0 +L
        key = 0
    else:
        a = x-r_0
        b = x+r_0
        key = 1
        
    if key:
        index = np.where((arr>a)&(arr<b))[0]
    else:
        index = np.where((arr<a)|(arr>b))[0]
        
    return index
    

In [ ]:
import numpy as np
import illustris_python as il

basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
L = 14
N = L
i = 0
k = 0

result = {'ratio_axis':np.zeros(0),'RMS':np.zeros(0),'haloID':np.zeros(0,dtype=int),
          'pos':np.zeros((0,L,3)),'vel':np.zeros((0,L,3)),'central_gal_ID':np.zeros(0,dtype=int),
         'sat_ID':np.zeros((0,L),dtype=int)}

ID_G_iso = dict(np.load('ID_G_iso.npz'))['ID_G_iso']
for haloID in ID_G_iso:
    sat_class_pos,sat_class_vel,sat_class_ID,ID_firstSub,groupID = PosVel_no_FOF(haloID,basePath,snapNum,N=L,M=300)
    
    print(i)
    i+=1
    if len(sat_class_pos) == 1:
        print(k)
        
        k+=1
        continue
            
    RMS,mean,indexes = coherent_iso_fast(sat_class_pos,sat_class_vel,N)
    ratio,eigvec     = ratio_axis(sat_class_pos)
    
    result['ratio_axis']     = np.append(result['ratio_axis'],ratio)
    result['RMS']            = np.append(result['RMS'],RMS)
    result['haloID']         = np.append(result['haloID'],groupID)
    result['pos']            = np.append(result['pos'],sat_class_pos.reshape((1,L,3)),axis=0)
    result['vel']            = np.append(result['vel'],sat_class_vel.reshape((1,L,3)),axis=0)
    result['central_gal_ID'] = np.append(result['central_gal_ID'],ID_firstSub)
    result['sat_ID']         = np.append(result['sat_ID'],sat_class_ID.reshape(1,L),axis=0)
    
np.savez('/home/tnguser/data/tng50_no_FOF_14.npz', 
         ratio_axis=result['ratio_axis'],RMS=result['RMS'],haloID=result['haloID'],pos=result['pos'],
         vel=result['vel'],central_gal_ID=result['central_gal_ID'],sat_ID=result['sat_ID'])

## c/a and $\Delta_{k}$ criteria

In [ ]:
def Transform_pos_vel(sat_Trees,cen_Tree,SnapRedshift,snapNum):
    import numpy as np
    
    #SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    #fields = ['SnapNum','SubhaloPos','SubhaloVel','Group_R_Crit200']
    
    h = 0.677
    a = 1 / (1+SnapRedshift[snapNum])
    f2 = a / h
    Num_sats = len(sat_Trees)
    
    index_cen = np.where(cen_Tree['SnapNum']==snapNum)[0]
    
    pos_cen = cen_Tree['SubhaloPos'][index_cen,:].reshape(3,)
    vel_cen = cen_Tree['SubhaloVel'][index_cen,:].reshape(3,)
    
    pos_sat = np.zeros([0,3])
    vel_sat = np.zeros([0,3])
    
#     for i in np.arange(Num_sats):
#         index_sat = np.where(sat_Trees[i]['SnapNum']==snapNum)[0]
#         pos_sat = np.append(pos_sat,sat_Trees[i]['SubhaloPos'][index_sat,:].reshape(1,3),axis=0)
#         vel_sat = np.append(vel_sat,sat_Trees[i]['SubhaloVel'][index_sat,:].reshape(1,3),axis=0)
        
    for i in np.arange(Num_sats):
        index_sat = np.where(sat_Trees[i]['SnapNum']==snapNum)[0]
        try:
            pos_sat = np.append(pos_sat,sat_Trees[i]['SubhaloPos'][index_sat,:].reshape(1,3),axis=0)
            vel_sat = np.append(vel_sat,sat_Trees[i]['SubhaloVel'][index_sat,:].reshape(1,3),axis=0)
        except ValueError:
            continue
            
    Num_sats = len(pos_sat[:,0])
            
    
    for i in np.arange(Num_sats):
        for j in np.arange(3):
            d = pos_sat[i,j]-pos_cen[j]                    # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            pos_sat[i,j] = cDistance * f2                  # unit kpc
            vel_sat[i,j] = vel_sat[i,j] - vel_cen[j]       # unit km/s
            
        
    return pos_sat, vel_sat
    

In [ ]:
def Draw(cen_find_ID,sat_find_IDs,snapNum,basePath,haloID,N):    
    import numpy as np 
    import illustris_python as il
    import matplotlib.pyplot as plt
    from matplotlib import animation

    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']

    sat_Trees   = []
    fields_cen = ['SnapNum','SubhaloPos','SubhaloVel','Group_R_Crit200']
    fields_sat = ['SnapNum','SubhaloPos','SubhaloVel']

    cen_Tree = il.sublink.loadTree(basePath,snapNum,id=cen_find_ID,fields=fields_cen,
                                           onlyMPB=True)
    for Id in sat_find_IDs:
        sat_Trees.append(il.sublink.loadTree(basePath,snapNum,id=Id,fields=fields_sat,
                                           onlyMPB=True))


    #fig, ax = plt.subplots()
    
    snapNums = [99]
    RMSs_co  = np.zeros(0)
    ratios    = np.zeros(0)
    redshifts= np.zeros(0)
    Snaps    = np.zeros(0)
    
    for num in snapNums:
        sat_class_pos, sat_class_vel = Transform_pos_vel(sat_Trees,
                                                         cen_Tree,SnapRedshift,snapNum=num)
        if len(sat_class_pos[:,0]) < len(sat_find_IDs):
            continue

        RMS, mean, index = coherent_iso_fast(sat_class_pos,sat_class_vel,N)
        ratio, min_vec   = ratio_axis(sat_class_pos)
        #print(index)
        RMSs_co   = np.append(RMSs_co,RMS)
        ratios     = np.append(ratios,ratio)
        Snaps     = np.append(Snaps,num)
        redshifts = np.append(redshifts,SnapRedshift[num])
        
    
    return Snaps,ratios,RMSs_co

In [ ]:
def sample(address,N):    
    import numpy as np
    import illustris_python as il
    Data = dict(np.load(address))
    basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
    snapNum  = 99
    
    HaloID       = Data['GroupID']
    SatID        = Data['Sat_IDs']
#     Snaps_infall = Data['snap_infalls']
    
    Snaps      = np.zeros(0,dtype=int)
    HaloID_now = np.zeros(0,dtype=int)
    Ratios     = np.zeros(0)
    RMSs    = np.zeros(0)
    
    Num = len(HaloID)
    
    for i in np.arange(Num):
        FoFGroup = il.groupcat.loadSingle(basePath, snapNum, haloID=HaloID[i])
        cen_find_ID = FoFGroup['GroupFirstSub']
        sat_class_ID= SatID[i]
        haloID      = HaloID[i]
#         snap_infall = Snaps_infall[i]
#         if snap_infall == -1:
#             continue
        snap,ratio,RMS = Draw(cen_find_ID,sat_class_ID,snapNum,basePath,haloID,N)
        
        Snaps = np.append(Snaps,snap)
        HaloID_now = np.append(HaloID_now,np.repeat(haloID,len(snap)))
        Ratios = np.append(Ratios,ratio)
        RMSs = np.append(RMSs,RMS)
        
    np.savez('/home/tnguser/data/sample_%d_abs_2.npz' %N,Snaps=Snaps,HaloID_now=HaloID_now,Ratios=Ratios,RMSs=RMSs)
    
    
return


### claculate c/a $\Delta_{k}$

In [ ]:
import numpy as np
address = '/home/tnguser/data/infall_time_14.npz'
number = np.arange(11) + 4
for N in number:
    sample(address,N)

### select systems matching c/a $\Delta_{k}$ criteria

In [ ]:
def dis_arr2d(arr,p=2,m=50,q=98):
    import numpy as np
    
    Len = len(arr[:,0])
    low  = np.percentile(arr,p,axis=1).reshape(1,Len)
    med  = np.percentile(arr,m,axis=1).reshape(1,Len)
    high = np.percentile(arr,q,axis=1).reshape(1,Len)
    result = np.concatenate((low,med,high),axis=0)
    return result

In [ ]:
def fun(Group,desp_high,ratio_high):
    from functools import reduce
    index = reduce(np.intersect1d,
                   [np.where(Group['Ratios']<ratio_high),
                    np.where((Group['RMSs']<=desp_high)&(Group['RMSs']>0))])

    return set(zip(Group['HaloID_now'][index],Group['Snaps'][index]))

In [ ]:
def set_N(obs_RMS,N,ratio_high):
    Groups  = dict(np.load('/home/tnguser/data/sample_%d_abs_2.npz' %N))
    
    index = N-4
    return fun(Groups,obs_RMS[2,index],ratio_high)



In [ ]:
import numpy as np
obs_RMS = dict(np.load('/home/tnguser/data/RMS_N_iso_fast_no_abs_14_1000.npz'))

obs_RMS = dis_arr2d(obs_RMS['RMS_N'],q=84)
ratio_high = 0.26

SET = set_N(obs_RMS,4,ratio_high)
number = [5,6,7,8,9,10,11,12,13,14]

for N in number:
    Set = set_N(obs_RMS,N,ratio_high)
    SET = SET.intersection(Set)
    